In [1]:
import os
import csv
import librosa
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import IPython.display as ipd
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy.signal import peak_widths
import math
from sklearn.model_selection import train_test_split
from scipy.sparse.csgraph import min_weight_full_bipartite_matching
from scipy.sparse import csr_matrix
from scipy.optimize import minimize
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
from tqdm import tqdm

2024-07-25 10:41:28.468618: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 10:41:29.024548: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-25 10:41:29.816496: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
seed_value = 1
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)
tf.keras.utils.set_random_seed(seed_value)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
folds_file = './ICBHI_Dataset/patient_list_foldwise.txt'
# train_flag = train_flag

In [ ]:
data_dir = '/home/monetai2/Desktop/LabFolder/Kim/LungSound/V5_mspec_8s_4gr/ICBHI_Dataset/audio_and_txt_files/'
# file_name = './Dataset/audio_and_txt_files/'
def Extract_Annotation_Data(file_name, data_dir):
	tokens = file_name.split('_')
	recording_info = pd.DataFrame(data = [tokens], columns = ['Patient Number', 'Recording index', 'Chest location','Acquisition mode','Recording equipment'])
	recording_annotations = pd.read_csv(os.path.join(data_dir, file_name + '.txt'), names = ['Start', 'End', 'Crackles', 'Wheezes'], delimiter= '\t')
	return recording_info, recording_annotations

# get annotations data and filenames
def get_annotations(data_dir):
	filenames = [s.split('.')[0] for s in os.listdir(data_dir) if '.txt' in s]
	i_list = []
	rec_annotations_dict = {}
	for s in filenames:
		i,a = Extract_Annotation_Data(s, data_dir)
		i_list.append(i)
		rec_annotations_dict[s] = a

	recording_info = pd.concat(i_list, axis = 0)
	recording_info.head()

	return filenames, rec_annotations_dict

def slice_data(start, end, raw_data, sample_rate):
	max_ind = len(raw_data) 
	start_ind = min(int(start * sample_rate), max_ind)
	end_ind = min(int(end * sample_rate), max_ind)
	return raw_data[start_ind: end_ind]

# def get_label(crackle, wheeze):
# 	if crackle == 0 and wheeze == 0:
# 		return 0
# 	else:
# 		return 1
def get_label(crackle, wheeze):
	if crackle == 0 and wheeze == 0:
		return 0
	elif crackle == 1 and wheeze == 0:
		return 1
	elif crackle == 0 and wheeze == 1:
		return 2
	else:
		return 3

def get_sound_samples(recording_annotations, file_name, data_dir, sample_rate):
	sample_data = [file_name]
	# load file with specified sample rate (also converts to mono)
	data, rate = librosa.load(os.path.join(data_dir, file_name+'.wav'), sr=sample_rate)
	#print("Sample Rate", rate)
	
	for i in range(len(recording_annotations.index)):
		row = recording_annotations.loc[i]
		start = row['Start']
		end = row['End']
		crackles = row['Crackles']
		wheezes = row['Wheezes']
		audio_chunk = slice_data(start, end, data, rate)
		sample_data.append((audio_chunk, start,end, get_label(crackles, wheezes)))
	return sample_data
filenames, rec_annotations_dict = get_annotations(data_dir)
    

In [ ]:
sample_rate = 4000
filenames_with_labels = []
print("Exracting Individual Cycles")
cycle_list = []
# classwise_cycle_list = [[], []] #2-class
classwise_cycle_list = [[], [], [], []] #4-class
for idx, file_name in tqdm(enumerate(filenames)):
    data = get_sound_samples(rec_annotations_dict[file_name], file_name, data_dir, sample_rate)
    # print('--------', data)
    cycles_with_labels = [(d[0], d[3], file_name, cycle_idx, d[3]) for cycle_idx, d in enumerate(data[1:])] #lable: d[3]
    # cycles_with_labels = [(d[0], d[3]) for cycle_idx, d in enumerate(data[1:])] #lable: d[3]
    # print('cycles_with_labels: ', cycles_with_labels)
    cycle_list.extend(cycles_with_labels)
    for cycle_idx, d in enumerate(cycles_with_labels):
        filenames_with_labels.append(file_name+'_'+str(d[3])+'_'+str(d[1]))
        classwise_cycle_list[d[1]].append(d)
print(len(cycle_list))
print(len(classwise_cycle_list))

In [ ]:
# augment normal
seed_value = 1
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)
tf.keras.utils.set_random_seed(seed_value)
scale = 1
aug_nos = scale*len(classwise_cycle_list[0]) - len(classwise_cycle_list[0])
for idx in range(aug_nos):
    # normal_i + normal_j
    i = random.randint(0, len(classwise_cycle_list[0])-1)
    j = random.randint(0, len(classwise_cycle_list[0])-1)
    normal_i = classwise_cycle_list[0][i]
    normal_j = classwise_cycle_list[0][j]
    new_sample = np.concatenate([normal_i[0], normal_j[0]])
    cycle_list.append((new_sample, 0, normal_i[2]+'-'+normal_j[2], idx, 0))
    filenames_with_labels.append(normal_i[2]+'-'+normal_j[2]+'_'+str(idx)+'_0')
    
# augment abnormal
aug_nos = scale*len(classwise_cycle_list[0]) - len(classwise_cycle_list[1])
for idx in range(aug_nos):
    aug_prob = random.random()
    if aug_prob < 0.6:
        # crackle_i + crackle_j
        i = random.randint(0, len(classwise_cycle_list[1])-1)
        j = random.randint(0, len(classwise_cycle_list[1])-1)
        sample_i = classwise_cycle_list[1][i]
        sample_j = classwise_cycle_list[1][j]
    elif aug_prob >= 0.6 and aug_prob < 0.8:
        # crackle_i + normal_j
        i = random.randint(0, len(classwise_cycle_list[1])-1)
        j = random.randint(0, len(classwise_cycle_list[0])-1)
        sample_i = classwise_cycle_list[1][i]
        sample_j = classwise_cycle_list[0][j]
    else:
        # normal_i + crackle_j
        i = random.randint(0, len(classwise_cycle_list[0])-1)
        j = random.randint(0, len(classwise_cycle_list[1])-1)
        sample_i = classwise_cycle_list[0][i]
        sample_j = classwise_cycle_list[1][j]

    new_sample = np.concatenate([sample_i[0], sample_j[0]])
    cycle_list.append((new_sample, 1, sample_i[2]+'-'+sample_j[2], idx, 0))
    filenames_with_labels.append(sample_i[2]+'-'+sample_j[2]+'_'+str(idx)+'_1')
print(len(cycle_list))

In [ ]:
# print(cycle_list[1:])
# print(cycle_list[0:10])
# for c in enumerate(cycle_list[1:10]):
#     print(c[1])
#     print(c[1][1])

cycle_list_new_data = []
cycles_with_labels_new = [(c[1][0], c[1][1]) for c in enumerate(cycle_list[0:])]
cycle_list_new_data.extend(cycles_with_labels_new)
print(len(cycle_list_new_data))


In [ ]:
import numpy as np
import tensorflow as tf

# Define the desired sequence length
max_sequence_length = 3*sample_rate

# Define your dense layer for data compression
dense_layer = tf.keras.layers.Dense(units=max_sequence_length, activation='relu')

# Separate data and labels
X = np.array([item[0] for item in cycle_list_new_data], dtype=object)
y = np.array([item[1] for item in cycle_list_new_data])

# Initialize X_new list to store modified sequences
X_new = []

for x in X:
    if len(x) > max_sequence_length:
        t = max_sequence_length // len(x)
        if max_sequence_length % len(x) == 0:
            repeat_x = np.tile(x, t)
            copy_repeat_x = repeat_x.copy()
            X_new.append(copy_repeat_x)
        else:
            d = max_sequence_length % len(x)
            # print('ddddd', d)
            d = x[:d]
            # print('dddddddd: ', d)
            # print('soundclip*t:', len(np.tile(soundclip, t)), n_samples*t)
            repeat_x = np.concatenate((np.tile(x, t), d))
            copy_repeat_x = repeat_x.copy()
            # print('copy_repeat_sample:', len(copy_repeat_sample))
            X_new.append(copy_repeat_x)
    else:
        padded_sequence = np.pad(x, (0, max_sequence_length - len(x)), 'constant')
        X_new.append(padded_sequence)
        # print('Smaller:', padded_sequence.shape)

# Convert to NumPy arrays
X_new = np.array(X_new)
y = np.array(y)
X = X_new
print(X.shape)

In [ ]:
from keras.layers import Input, Conv1D, MaxPooling2D, Flatten, Dense, Concatenate, GlobalAveragePooling1D, Activation, Add, AveragePooling1D, ReLU
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.losses import categorical_crossentropy
from keras.metrics import binary_accuracy, Precision, Recall
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

def residual_block(x, filters, dilation_rate):
    # Define the residual block
    res = Conv1D(filters=filters, kernel_size=3, dilation_rate=dilation_rate, padding='same')(x)
    res = BatchNormalization()(res)
    res = Activation('relu')(res)
    res = Conv1D(filters=filters, kernel_size=3, padding='same')(res)
    res = BatchNormalization()(res)
    res = Add()([x, res])
    res = Activation('relu')(res)
    return res

feature_size = max_sequence_length
# Define the input shape
input_shape = (feature_size, 1) 

# Define the input layer
inputs = Input(shape=input_shape)


x = Conv1D(64, kernel_size=3, padding='same')(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.1)(x)
x = AveragePooling1D(pool_size=2)(x)

x = Conv1D(128, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.15)(x)
x = AveragePooling1D(pool_size=2)(x)

x = Conv1D(256, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.2)(x)

x = Conv1D(256, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.2)(x)
x = AveragePooling1D(pool_size=2)(x)

x = Conv1D(512, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.25)(x)

x = Conv1D(512, kernel_size=3, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.25)(x)


# Define the TCN layers
dilation_bases = [2, 3]
# dilation_factors = [1, 2, 4]
tcn_outputs = []
for dilation_base in dilation_bases:
    for i in range(3):
        dilation_rate = dilation_base**i
        x = residual_block(x, filters=512, dilation_rate=dilation_rate)
    tcn_outputs.append(x)

# Define the fusion layer
fusion = Concatenate(axis=-1)(tcn_outputs)
fusion = GlobalAveragePooling1D()(fusion)

# Define the classifier
x = Dense(units=512, activation='relu')(fusion)
# x = Dropout(0.5)(x)
x = Dense(units=128, activation='relu')(x)
# x = Dropout(0.5)(x)
x = Dense(units=32, activation='relu')(x)
outputs = Dense(units=4, activation='softmax')(x)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), 
                       tf.keras.metrics.Recall()]) #[binary_accuracy, Precision(), Recall()]

# Print the model summary
model.summary()

In [ ]:
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from keras.applications.vgg19 import VGG19, preprocess_input
import numpy as np

# Preprocess the image data
best_test_acc = 0.0
best_model = None
sp_scores, se_scores, sc_scores = [], [], []
ppv, se_class, f1, sp_class, acc = [], [], [], [], []

for i in range(5):
    # Split the dataset into training, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True,test_size=0.4, random_state=42, stratify=y)
    
    unique, frequency = np.unique(y_test, return_counts = True)
    print(["{:0.3f}".format(fre/len(y_test)) for fre in frequency])
    # Preprocess the image data
    X_train = preprocess_input(X_train)
    X_test = preprocess_input(X_test)
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    y_train = to_categorical(y_train, num_classes=4)
    y_test = to_categorical(y_test, num_classes=4)

    # model = build_model(input_shape)
    # model = Model(inputs=base_model.input, outputs=outputs)
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='categorical_crossentropy',
                metrics=['accuracy', tf.keras.metrics.Precision(), 
                        tf.keras.metrics.Recall()]) 
    model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test), callbacks=[callback])
    test_metrics = model.evaluate(X_test, y_test)
    test_acc = test_metrics[1]
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_model = model
    best_model.save(f'./model_seed/LN_ML_TCN/60_40/Sequence/Dup_Trun/model_Aug_{i}.h5')

        # Compute the confusion matrix
    y_test = np.argmax(y_test, axis = 1)
    y_pred = best_model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    # clas_report = classification_report(y_val, y_pred)
    avg_cm = confusion_matrix(y_test, y_pred)

    se = (avg_cm[1][1] + avg_cm[2][2] + avg_cm[3][3])/(avg_cm[1][0] + avg_cm[1][1] + avg_cm[1][2] + avg_cm[1][3] 
                                                    + avg_cm[2][0] + avg_cm[2][1] + avg_cm[2][2] + avg_cm[2][3]
                                                    + avg_cm[3][0] + avg_cm[3][1] + avg_cm[3][2] + avg_cm[3][3])
    sp = avg_cm[0][0]/(avg_cm[0][0] + avg_cm[0][1] + avg_cm[0][2] + avg_cm[0][3])
    sc = (se+sp)/2
    sp_scores.append(sp)
    se_scores.append(se)
    sc_scores.append(sc)
    # print(f"Fold {i+1}: Sp={sp:.4f}, Se={se:.4f}, Sc={sc:.4f}")
    # Reset the model for the next fold
    tf.keras.backend.clear_session()

    # Caclulate the value of each class
    # confusion_matrix = avg_cm
    num_classes = avg_cm.shape[0]

    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)
    f1_sc = np.zeros(num_classes)
    accuracy = np.zeros(num_classes)
    specificity = np.zeros(num_classes)

    for j in range(num_classes):
        true_positive = avg_cm[j, j]
        false_positive = np.sum(avg_cm[:, j]) - true_positive
        false_negative = np.sum(avg_cm[j, :]) - true_positive
        true_negative = np.sum(avg_cm) - (true_positive + false_positive + false_negative)

        accuracy[j] = (true_positive + true_negative) / np.sum(avg_cm)
        specificity[j] = true_negative / (true_negative + false_positive)

        precision[j] = true_positive / (true_positive + false_positive)
        recall[j] = true_positive / (true_positive + false_negative)
        f1_sc[j] = 2 * (precision[j] * recall[j]) / (precision[j] + recall[j])

    ppv.append(precision)
    se_class.append(recall)
    f1.append(f1_sc)
    sp_class.append(specificity)
    acc.append(accuracy)

# Print overall scores
print("\nAverage scores:")
print(f"Sp: {np.mean(sp_scores):.4f}")
print(f"Se: {np.mean(se_scores):.4f}")
print(f"Sc: {np.mean(sc_scores):.4f}")

In [ ]:
# Save the results in a .txt file
with open('./model_seed/LN_ML_TCN/60_40/Sequence/Dup_Trun/Mean_results_Aug.txt', 'w') as file:
    file.write('Specificity Sp: {}\n'.format(np.mean(sp_scores)))
    file.write('Sensitivity Se: {}\n'.format(np.mean(se_scores)))
    file.write('Score Sc: {}\n'.format(np.mean(sc_scores)))
    file.write('confusion_matrix for each class: {}\n'.format(confusion_matrix))

# Save the results in a .txt file
with open('./model_seed/LN_ML_TCN/60_40/Sequence/Dup_Trun/Each_fold_results_Aug.txt', 'w') as file:
    file.write('Specificity Sp: {}\n'.format(sp_scores))
    file.write('Sensitivity Se: {}\n'.format(se_scores))
    file.write('Score Sc: {}\n'.format(sc_scores))

# Save the results in a .txt file
with open('./model_seed/LN_ML_TCN/60_40/Sequence/Dup_Trun/Each_class_results_Aug.txt', 'w') as file:
    file.write('PPv: {}\n'.format(ppv))
    file.write('Se: {}\n'.format(se_class))
    file.write('F1: {}\n'.format(f1))
    file.write('Sp_class: {}\n'.format(sp_class))
    file.write('Acc: {}\n'.format(acc))    